# Project: LLMs in Education (CrewAI)
---
### Team 8: Gunjan Sharma, Jasmine Gohil, Jenil Shah, Saachi Dholakia, Sneha Ekka

In this notebook we explore the application of large language models (LLMs) in education, focusing on how CrewAI, utilizing OpenAI as its base model, performs in generating responses for different educational prompts by creating an Agent that acts as an AI tutor for students. We analyze the quality and relevance of the outputs across three specific prompts.


## Installing & Importing Libraries

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
!pip install utils

from crewai import Agent, Task, Crew
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.6/339.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

## Connecting to OpenAI API

In [ ]:
!pip install python-dotenv

from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access API key from environment variable
openai_api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY not found in environment variables. Please create a .env file with your key.")

In [ ]:
# Fetch your OpenAI API Key
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## CrewAI Agentic LLM
---
In this model, we define three distinct agents, each with their own specific roles, goals, and tasks. These agents work collaboratively to enhance the learning experience by collecting contextual information, specifying relevant subjects, and providing detailed assistance on specific topics. Below are the descriptions of each agent:

- **Agent 1:** Student Context Collector is responsible for gathering comprehensive information about the student's current context. This includes understanding their educational background, learning preferences, and current knowledge gaps.
- **Agent 2:** Subject Topic Specifier is tasked with defining the subjects and topics that are relevant to the student's current educational needs. This agent uses the information collected by the Student Context Collector to tailor the learning material.
- **Agent 3:** Topic Helper provides detailed assistance on specific topics identified by the Subject Topic Specifier. This agent is responsible for delivering educational content, answering questions, and offering explanations to facilitate learning.


In [ ]:
# Creating Agents
student_context_collector = Agent(
    role='Collect Student Context from {subject_topic}',
    goal='Ask the student about their education level, learning disabilities, and any specific needs related to their study of {subject_topic}',
    backstory=(
        'You are a friendly and understanding tutor. '
        'You will ask the student about their educational background, any learning disabilities, and specific needs '
        'to better understand how to assist them with their studies in the given subject topic.'
    ),
    allow_delegation=False,
    verbose=True
)

subject_topic_specifier = Agent(
    role='Specify Subject Topic act as a tutor for the specified subject from {subject_topic}',
    goal='Determine the specific area within the subject of {subject_topic} that the student needs help with',
    backstory=(
        'You are a knowledgeable tutor. '
        'You will ask the student to specify the particular area within the subject topic they need help with, '
        'ensuring a focused and effective tutoring session.'
    ),
    allow_delegation=False,
    verbose=True
)

topic_helper = Agent(
    role='Provide Help on Specific Topic as a tutor from {subject_topic}',
    goal='Provide detailed assistance on the specific {subject_topic} area based on the student\'s context and needs',
    backstory=(
        'You are an expert tutor in the specified subject area. '
        'Based on the student\'s educational background and specific needs, '
        'you will provide tailored assistance and explanations to help them understand and excel in the specific area they are struggling with.'
    ),
    allow_delegation=False,
    verbose=True
)

# Creating Tasks
collect_context = Task(
    description=(
        '1. Ask the student about their education level.\n'
        '2. Inquire about any learning disabilities or specific needs they might have.\n'
        '3. Understand the context to provide better-targeted assistance.'
    ),
    expected_output='Student\'s education level, learning disabilities, and specific needs',
    agent=student_context_collector
)

specify_topic = Task(
    description=(
        '1. Ask the student to specify the particular area within the subject topic they need help with.\n'
        '2. Ensure the area is clearly defined to provide focused assistance.'
    ),
    expected_output='The specific area within the subject topic that the student needs help with',
    agent=subject_topic_specifier
)

provide_help = Task(
    description=(
        '1. Provide detailed assistance on the specific subject area based on the student\'s context and needs.\n'
        '2. Offer explanations, examples, and any additional resources to aid understanding.\n'
        '3. Tailor the assistance to the student\'s educational background and specific requirements.'
    ),
    expected_output='Detailed assistance and explanations for the specified subject area',
    agent=topic_helper
)

crew = Crew(
    agents=[student_context_collector, subject_topic_specifier, topic_helper],
    tasks=[collect_context, specify_topic, provide_help],
    verbose=2
)

## Prompt 1
---
**Topic:** Accounting Principles

In [ ]:
# Define the inputs with correct structure
inputs = {
    'subject_topic': 'accounting principles',
    'education_prompt': """
Plan a detailed study assistance session for a student needing help with accounting principles.
The student is currently in their second year of college and has a learning disability affecting their ability to understand complex mathematical concepts.
The session should cover basic accounting principles and include examples to aid understanding.
"""
}

# Use the Crew to process the education prompt
result = crew.kickoff(inputs=inputs)

# Print the resulting assistance plan
print("Your study assistance plan is ready:")
print(result)

 [DEBUG]: == Working Agent: Collect Student Context from accounting principles
 [INFO]: == Starting Task: 1. Ask the student about their education level.
2. Inquire about any learning disabilities or specific needs they might have.
3. Understand the context to provide better-targeted assistance.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 
Student's education level: I would like to know more about your educational background. Can you tell me about your current level of education and any previous experience you have with accounting principles?

Learning disabilities and specific needs: Do you have any learning disabilities or specific needs that I should be aware of in order to better assist you with your studies in accounting principles?

By understanding your education level, learning disabilities, and specific needs, I will be able to provide you with targeted assistance and support to help you succeed in your study of accounting principle

### Insights

The LLM's response was exceptionally well-targeted, focusing on a specific accounting principle and providing practical examples that are crucial for understanding these concepts in a real-world context. The generated content closely matched the requirements of the prompt, demonstrating excellent alignment with the specified needs. The assistance plan was both detailed and specific, ensuring that it effectively addressed the student's requirements. By incorporating clear and practical examples, the LLM not only provided comprehensive information but also made it accessible and applicable, thereby significantly enhancing the student's understanding of the accounting principles discussed. This detailed and focused approach made the assistance plan particularly effective for the student's learning objectives.

## Prompt 2
---
**Topic:** War in Ukraine

In [ ]:
inputs = {
    'subject_topic': 'current affairs about the war in Ukraine',
    'education_prompt': """
Plan a detailed study assistance session for a student needing help with understanding current affairs about the war in Ukraine.
The student is currently in their second year of college and has a learning disability affecting their ability to understand complex political and historical concepts.
The session should cover the historical background, key events, major players, and the impact of the conflict on global politics and economy, using simplified explanations and relevant examples to aid understanding.
"""
}

# Use the Crew to process the education prompt
result = crew.kickoff(inputs=inputs)

# Print the resulting assistance plan
print("Your study assistance plan is ready:")
print(result)

 [DEBUG]: == Working Agent: Collect Student Context from current affairs about the war in Ukraine
 [INFO]: == Starting Task: 1. Ask the student about their education level.
2. Inquire about any learning disabilities or specific needs they might have.
3. Understand the context to provide better-targeted assistance.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 
In order to provide the best assistance with your studies on the current affairs about the war in Ukraine, I would like to understand more about your educational background. Can you please share with me your education level? Additionally, it would be helpful for me to know if you have any learning disabilities or specific needs that I should be aware of in order to better support you in this subject area. Thank you for sharing this information with me so that I can tailor my assistance to meet your individual needs.

> Finished chain.
 [DEBUG]: == [Collect Student Context from current af

### Insights

The LLM successfully provided a comprehensive overview, addressing all aspects of the prompt in a broad and detailed manner. The study assistance plan was well-aligned with the specified requirements, ensuring that it met the needs outlined in the prompt. The response was both extensive and thorough, effectively covering the complexity of the topic. By encompassing the full scope of the prompt, the LLM demonstrated a robust understanding and delivered a well-rounded and detailed analysis, making the assistance plan highly effective and relevant.

## Prompt 3
---
**Topic:** Fundamentals of Environmental Science

In [ ]:
inputs = {
    'subject_topic': 'the fundamentals of environmental science',
    'education_prompt': """
Plan a detailed study assistance session for a student needing help with understanding the fundamentals of environmental science.
The student is currently in high school.
The session should cover the basic principles of environmental science, including ecosystems, biodiversity, pollution, and sustainable practices, using simplified explanations and relevant examples to aid understanding.
"""
}

# Use the Crew to process the education prompt
result = crew.kickoff(inputs=inputs)

# Print the resulting assistance plan
print("Your study assistance plan is ready:")
print(result)


 [DEBUG]: == Working Agent: Collect Student Context from the fundamentals of environmental science
 [INFO]: == Starting Task: 1. Ask the student about their education level.
2. Inquire about any learning disabilities or specific needs they might have.
3. Understand the context to provide better-targeted assistance.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 
For the fundamentals of environmental science, I would like to start by asking you about your education level. Can you please share with me your current educational background? 

Additionally, I would like to inquire if you have any learning disabilities or specific needs that I should be aware of to better assist you in your studies of environmental science. Understanding these factors will help me provide more targeted and effective support to help you succeed in this subject.

> Finished chain.
 [DEBUG]: == [Collect Student Context from the fundamentals of environmental science] Task

### Insights

The LLM provided comprehensive content on climate change, but it did not fully cover all the requested topics, leading to partial alignment with the overall objectives. While the information on climate change was thorough and detailed, it did not encompass the broader range of environmental topics specified in the original prompt. This resulted in a focused but narrow scope that limited the LLM's ability to fully address the entire spectrum of environmental issues. Although the depth of information on climate change was impressive and demonstrated a solid understanding of that particular topic, the lack of breadth meant that other important environmental subjects were not adequately covered. Consequently, while the LLM excelled in providing detailed insights into climate change, it fell short in offering a comprehensive overview of the broader environmental context.

## Conclusion
---
Comparing the quality of outputs for the three prompts reveals distinct differences in performance. CrewAI with OpenAI as the base model delivered the best results for the accounting principles prompt, providing a well-targeted, detailed, and practical response that closely matched the prompt's requirements. The moderate result was seen in the war in Ukraine prompt, where the response was thorough but lacked breadth, leading to partial alignment with the specified needs. The least effective performance was observed in the environmental science prompt, which, despite being broad and detailed, lacked the necessary depth in certain areas to fully address the complexity of the topic. Overall, the model excelled in delivering focused and practical assistance for accounting principles but struggled to provide comprehensive coverage for broader and more complex environmental topics.